In [49]:
import cupy as cp
import numpy as np
import asyncio
import hashlib
import aiohttp
import requests
import json

In [60]:

class API:
    base = "https://recruitment-test.investcloud.com/api/numbers"
    cache = {}
    rows = []
    col = []
    result = []

    def __init__(self, size):
        self.size = size

    async def __aenter__(self):
        self.session = aiohttp.ClientSession()
        return self
    
    async def __aexit__(self, exc_type, exc, tb):
        await self.session.close()


    async def initialize(self):
        async with self.session.get(f"{self.base}/init/{self.size}") as response:
            if response.status != 200:
                await self.session.close()
                raise aiohttp.ClientResponseError(response.request_info, response.history, code=response.status)
            
    async def _get_data(self, matrix: str, axis: str, idx: int) -> list[int]:
        if matrix != "A" and matrix != "B":
            raise ValueError("Invalid matrix")
        if axis != "row" and axis != "col":
            raise ValueError("Invalid axis")
        if idx < 0 or idx >= self.size:
            raise ValueError("Invalid column")
        
        url = f"{self.base}/{matrix}/{axis}/{idx}"
        async with self.session.get(url) as response:
            data = await response.content.read()
            return self.parse_list(data)
    
    async def main(self, strategy: int = 0):
        result = cp.zeros((self.size, self.size))

        await self.initialize()

        row_tasks = [self._get_data("A", "row", row) for row in range(self.size)]
        col_tasks = [self._get_data("B", "col", col) for col in range(self.size)]

        self.rows, self.cols = await asyncio.gather(asyncio.gather(*row_tasks), asyncio.gather(*col_tasks))

        self.rows = cp.array(self.rows)
        self.cols = cp.array(self.cols).T
        self.result = cp.matmul(self.rows, self.cols)

        self.validate(self.get_hash(*result, strategy))

        await self.session.close()

        print({
            "hash": self.get_hash(result, strategy),
        })
    
    def get_hash(self, *solution, strategy: int = 0):
        
        if strategy == 0:
            transpose = np.transpose(solution)
            flatten = "".join([str(i) for i in transpose.flatten()])
            return hashlib.md5(flatten.encode()).hexdigest()
        elif strategy == 1:
            flatten = "".join([str(i) for i in solution.flatten()])
            return hashlib.md5(flatten.encode()).hexdigest()
        elif strategy == 2:
            flatten = ""
            buffer = bytearray()
            for row in range(self.size):
                for col in range(self.size):
                    flatten += str(solution[row][col])
                    buffer.append(int(solution[row][col]))
            
            print(buffer)
            print(hashlib.md5(buffer, usedforsecurity=False))
            return hashlib.md5(buffer, usedforsecurity=False)
    @staticmethod
    def parse_list(data: bytes) -> list[int]:
        start = data.find(b"'Value': [") + len(b"'Value': [") + 1
        end = data.find(b"]", start)
        value_b = data[start:end]
        return list(map(int, value_b.split(b",")))
    
    def validate(self, hash):
        header = {
            "Content-Type": "text/json; charset=utf-8",
            
        }

        payload = json.dumps(hash)
        print("Payload", payload)

        try:
            response = requests.post(f"{self.base}/validate", headers=header, json=hash)
            response.raise_for_status()
            if response.json()["Value"] == "Alas it didn't work":
                print("Incorrect!", response.json())
            else:
                print("Correct!", response.json())
        except:
            raise Exception("Failed to validate matrix")
        

In [53]:
async with API(1000) as api:
    await api.main(strategy=2)

ValueError: byte must be in range(0, 256)

In [58]:
api = API(10)
await api.__aenter__()
await api.main(strategy=0)


Payload "bd9b53444bbdf5be62433c895dc86b03"
Incorrect! {'Value': "Alas it didn't work", 'Cause': None, 'Success': True}
{'hash': 'bd9b53444bbdf5be62433c895dc86b03'}


In [59]:
api.result

AttributeError: 'API' object has no attribute 'result'

In [ ]:
await api.__aexit__(None, None, None)

In [ ]:
import requests

url = "https://recruitment-test.investcloud.com/api/numbers/A/row/0"
data = requests.get(url).json()["Value"]
api_row = api.rows[0]

for x, y in zip(data, api_row):
    if x != y:
        print(x, y)